In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import tensorflow as tf
from tensorflow_addons.optimizers.utils import fit_bn

from train import *

gcs_path = 'gs://hm-eu-w4/jigsaw/t12/20200429_202612_t8c'
params = tf.io.gfile.glob(f'{gcs_path}/params*.csv')
print(params)
params = params[0]
params = pd.read_csv(params)

kw_params = params.T[0].to_dict()
print(params.T)
gc.collect()

# random.seed(seed)
# np.random.seed(seed)
# tf.random.set_seed(seed)
# os.environ['PYTHONHASHSEED'] = str(seed)
# os.environ['TF_DETERMINISTIC_OPS'] = '1'

strategy = setup_tpu(None)

## Configuration
#     path = f'{path}/{time.strftime("%Y%m%d_%H%M%S")}_{tpu_id}'
# gcs_path = f'gs://{gcs}/{path}'
checkpoint_path = f"{gcs_path}/best_model.tf"
print('gcs_path:', gcs_path)
# params['gcs_path'] = gcs_path

## Load and Train
with strategy.scope():
    model = build_model(**kw_params)
#     model, preds, sub = train_model(model, strategy, checkpoint_path, **kw_params)

#     ## Save results
#     plot_history(model.history, path, gcs)
#     history = pd.DataFrame(model.history.history)
#     print(history)
#     history.to_csv(f'{gcs_path}/history.csv', index=False)

# load best
# latest = tf.train.latest_checkpoint(checkpoint_dir)
checkpoint_path = f'{gcs_path}/best_model.tf'
with strategy.scope():
    model.load_weights(checkpoint_path)
    model.save_weights(f'../xlm-r_{"_".join(gcs_path.split("/")[-3:])}_{params["auc"].values[0]}.h5')


['gs://hm-eu-w4/jigsaw/t12/20200429_202612_t8c/params0.962192.csv']
                                                                 0
tpu_id                                                         t8c
max_len                                                        192
seed                                                            88
path                                                    jigsaw/t12
gcs                                                       hm-eu-w4
optimizer                                                     LAMB
batch_size                                                      27
lr                                                     9.25582e-05
weight_decay                                           4.80997e-05
dropout                                                        0.3
loss_fn                                                        bce
label_smoothing                                          0.0328898
warm_up                                                    3.

NameError: name 'batch_size' is not defined

In [7]:

batch_size = 1 #params["batch_size"].values[0] * strategy.num_replicas_in_sync
print('batch_size:', batch_size)

valid_dataset, test_dataset = val_np_dataset(batch_size=batch_size)

""" load npz datasets """
dataset = '../input/jigsaw20-val-test-ds/jigsaw20_val_ds.npz'
array = np.load(dataset)
x_valid, x_test, y_valid = [array[k] for k in list(array)]
print(x_valid.shape, x_test.shape, y_valid.shape)

fit_bn(model, x_test, np.ones(len(x_test))*0.5)

preds = model.predict(valid_dataset, verbose=1)
# sub_y = model.predict(test_dataset, verbose=1)

## Load Dataset
comp_ds = '../input/jigsaw-multilingual-toxic-comment-classification'
valid = pd.read_csv(f'{comp_ds}/validation.csv')
#     test = pd.read_csv(f'{comp_ds}/test.csv')
sub = pd.read_csv(f'{comp_ds}/sample_submission.csv')

valid['pred'] = preds
valid.to_csv(f'{gcs_path}/valid_oof_test.csv', index=False)

valid.groupby('toxic').pred.hist(bins=100, log=True, alpha=0.5)
plt.legend([0, 1])
save_fig('valid_hist_test.png', path, gcs)

valid[valid.toxic == 1].groupby('lang').pred.hist(bins=50, log=True, alpha=0.34)
plt.legend(valid.lang.unique())
save_fig('valid_toxic_hist_test.png', path, gcs)

valid_auc = roc_auc_score(valid.toxic, valid.pred)
print('AUC:', valid_auc,
        'toxic:', valid.toxic.mean(),
        'pred:', valid.pred.mean(),
        'ratio:', (valid.pred > 0.5).mean())

## Submission
sub['toxic'] = sub_y
sub.to_csv(f'{gcs_path}/submission_test.csv', index=False)

sub.toxic.hist(bins=100, log=True)
save_fig('sub_hist_test.png', path, gcs)
print('mean:', sub.toxic.mean(), 'ratio:', (sub.toxic > 0.5).mean())
#     assert sub.toxic.mean() < 0.3

## Save params
params['auc'] = valid_auc
# params.to_csv(f'{gcs_path}/params{valid_auc:04f}.csv', index=False)
print(params.T)

batch_size: 27
(8000, 192) (63812, 192) (8000,)
Train for 2364 steps
   1/2364 [..............................] - ETA: 1:22

IndexError: tuple index out of range

In [9]:
# import pdb; pdb.pm()

In [3]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
img = mpimg.imread('sub_hist.png')
_ = plt.imshow(img)
plt.show()
img = mpimg.imread('valid_hist.png')
_ = plt.imshow(img)
plt.show()
img = mpimg.imread('valid_toxic_hist.png')
_ = plt.imshow(img)

FileNotFoundError: [Errno 2] No such file or directory: 'sub_hist.png'

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
img = mpimg.imread('sub_hist_test.png')
_ = plt.imshow(img)
plt.show()
img = mpimg.imread('valid_hist_test.png')
_ = plt.imshow(img)
plt.show()
img = mpimg.imread('valid_toxic_hist_test.png')
_ = plt.imshow(img)